## Ensamble de modelos para predecir precios de acciones

### 1. Instalación de librerías

In [ ]:
# Librerías generales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import datetime as dt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

# Librerías LSTM
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.utils import plot_model
import tensorflow as tf

# Librerías Skforecast
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

### 2. Carga y preparación de datos

In [ ]:
# Parámetros para acceder lago de datos
silver = './data/silver/stock-prices/'
gold = './data/gold/portfolio-optimization/'
silver_table = 'stock_prices.csv'
gold_table = 'portfolio_optimization.csv'

# Parámetros del modelo
ticker = ['AAPL','MSFT','AMZN','TSLA','GOOGL','GOOG','NVDA','BRK-B','META','UNH'] 
metric_to_predict = 'Adj Close'
days_to_predict = 3 # Short-term future days to predict
pred_span_days = 60 # Set the number of days used for prediction
backward_steps = 180 # Set the backward steps to go from the last observation available
cpu_list = [i for i in ticker]
n_features = len(cpu_list)

# Lectura y transformación de datos
data = pd.read_csv(silver+silver_table)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
df_c = pd.pivot_table(data, values=metric_to_predict, columns=['Ticker'], index=data.index)
df_c = df_c[ticker]

# Construcción de modelo para imputar missing values
imputer = KNNImputer(n_neighbors=5, weights='uniform')

# Ajuste de modelo para imputar missing values por ticker
for i in ticker:
      imputer.fit(df_c[[i]])
      df_c[i] = imputer.transform(df_c[[i]]).ravel()

# Extracción de porción de datos a usar
df_train = df_c.iloc[-backward_steps:]
df_test = df_c.iloc[backward_steps:]

def inverse_transform(y):
 y_reshaped = y.reshape(-1, y.shape[-1])
 y_inverse = scaler.inverse_transform(y_reshaped)
 return y_inverse

def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)): 
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

def evaluate_forecast(y_test_inverse, yhat_inverse):
 mse_ = tf.keras.losses.MeanSquaredError()
 mae_ = tf.keras.losses.MeanAbsoluteError()
 mape_ = tf.keras.losses.MeanAbsolutePercentageError()
 mae = mae_(y_test_inverse,yhat_inverse)
 print('mae:', mae)
 mse = mse_(y_test_inverse,yhat_inverse)
 print('mse:', mse)
 mape = mape_(y_test_inverse,yhat_inverse)
 print('mape:', mape)

scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train[cpu_list])
scaled_test = scaler.transform(df_test[cpu_list])
scaled_pred = scaler.transform(df_c[cpu_list])

X_train, y_train = split_sequence(scaled_train, look_back=pred_span_days, forecast_horizon=days_to_predict)
X_test, y_test = split_sequence(scaled_test, look_back=pred_span_days, forecast_horizon=days_to_predict)
X_pred, y_pred = split_sequence(scaled_pred, look_back=pred_span_days, forecast_horizon=days_to_predict)

### 3. Entrenamiento de modelos

In [ ]:
# CNN-LSTM
def model2(units, dropout_rate):
    model_enc_dec_cnn = Sequential()
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=9, activation='relu', input_shape=(pred_span_days, n_features)))
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
    model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
    model_enc_dec_cnn.add(Flatten())
    model_enc_dec_cnn.add(RepeatVector(days_to_predict))
    model_enc_dec_cnn.add(LSTM(units, activation='relu', return_sequences=True))
    model_enc_dec_cnn.add(TimeDistributed(Dense(100, activation='relu')))
    model_enc_dec_cnn.add(TimeDistributed(Dense(n_features)))
    model_enc_dec_cnn.compile(loss='mse', optimizer='adam')
    return model_enc_dec_cnn

units = 128
epochs = 25
dropout_rate = 0.2
batch_size = 32
validation = 0.05

model_enc_dec_cnn = model2(128,0.2)
model_enc_dec_cnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation)
yhat = model_enc_dec_cnn.predict(X_test, verbose=0)
yhat_inverse = inverse_transform( yhat)
y_test_inverse = inverse_transform(y_test)
evaluate_forecast(y_test_inverse, yhat_inverse)
CNN_LSTM_train = yhat_inverse

In [ ]:
# Random forest
forecaster_reg = ForecasterAutoregMultiSeries(
                    regressor          = RandomForestRegressor(random_state=123, bootstrap=True, max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=10),
                    lags               = 7
                )
forecaster_reg.fit(df_train)
RDM_FRST_train = forecaster_reg.predict(days_to_predict)

In [ ]:
# XGBoost
forecaster_xgb = ForecasterAutoregMultiSeries(
                    regressor          = XGBRegressor(random_state=123, learning_rate=0.1, max_depth=3, max_iter=100),
                    lags               = 14
                )
forecaster_xgb.fit(df_train)
XGB_train = forecaster_xgb.predict(days_to_predict)

In [ ]:
# Ensamble

# Cambio de estructura de predicciones
CNN_LSTM_train = list(CNN_LSTM_train[-days_to_predict:].reshape(-1))
RDM_FRST_train = list(RDM_FRST_train.values.reshape(-1))
XGB_train = list(XGB_train.values.reshape(-1))
TEST_train = list(df_test[-days_to_predict:].values.reshape(-1))
preds = np.stack([
    CNN_LSTM_train, RDM_FRST_train, XGB_train
])
preds = pd.DataFrame(preds).transpose()

# Entrenamiento
stacker = LinearRegression()
stacker.fit(X=preds, y=TEST_train)

# Coeficientes del modelo ensamble
stacker.intercept_, stacker.coef_

# Predicción
STCK = stacker.predict(preds)
evaluate_forecast(TEST_train, STCK)

### 4. Predicción de datos

In [ ]:
# Funciones para predecir
def predict(num_prediction, model):
    prediction_list = data[-pred_span_days+1:][metric_to_predict]
    for _ in range(days_to_predict):
        yhat = model_enc_dec_cnn.predict(X_pred, verbose=0)
        yhat_inverse = inverse_transform(yhat)[0]
        prediction_list = np.append(prediction_list, yhat_inverse)
    prediction_list = prediction_list[pred_span_days-1:]
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = data.index[len(data.index)-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

In [ ]:
# CNN-LSTM
def model2(units, dropout_rate):
    model_enc_dec_cnn = Sequential()
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=9, activation='relu', input_shape=(pred_span_days, n_features)))
    model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=11, activation='relu'))
    model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
    model_enc_dec_cnn.add(Flatten())
    model_enc_dec_cnn.add(RepeatVector(days_to_predict))
    model_enc_dec_cnn.add(LSTM(units, activation='relu', return_sequences=True))
    model_enc_dec_cnn.add(TimeDistributed(Dense(100, activation='relu')))
    model_enc_dec_cnn.add(TimeDistributed(Dense(n_features)))
    model_enc_dec_cnn.compile(loss='mse', optimizer='adam')
    return model_enc_dec_cnn

units = 128
epochs = 25
dropout_rate = 0.2
batch_size = 32
validation = 0.05

model_enc_dec_cnn = model2(128,0.2)
model_enc_dec_cnn.fit(X_pred, y_pred, epochs=epochs, batch_size=batch_size, validation_split=validation)
forecast = predict(days_to_predict, model_enc_dec_cnn)
forecast_dates = predict_dates(days_to_predict)
CNN_LSTM = forecast

In [ ]:
# Random forest
forecaster_reg = ForecasterAutoregMultiSeries(
                    regressor          = RandomForestRegressor(random_state=123, bootstrap=True, max_depth=30, min_samples_leaf=2, min_samples_split=10, n_estimators=10),
                    lags               = 7
                )
forecaster_reg.fit(df_c)
RDM_FRST = forecaster_reg.predict(days_to_predict)

In [ ]:
# XGBoost
forecaster_xgb = ForecasterAutoregMultiSeries(
                    regressor          = XGBRegressor(random_state=123, learning_rate=0.1, max_depth=3, max_iter=100),
                    lags               = 14
                )
forecaster_xgb.fit(df_c)
XGB = forecaster_xgb.predict(days_to_predict)

In [ ]:
# Ensamble

# Cambio de estructura de predicciones
RDM_FRST = list(RDM_FRST.values.reshape(-1))
XGB = list(XGB.values.reshape(-1))
preds = np.stack([
    CNN_LSTM, RDM_FRST, XGB
])
preds = pd.DataFrame(preds).transpose()

# Predicción
STCK = stacker.predict(preds)
preds.rename(columns={0:'LSTM',1:'RF',2:'XGB'},inplace=True)
preds['Pred'] = STCK

t = []
for j in range(days_to_predict):
    t.append(cpu_list)

d = []
for i in cpu_list:
    d.append(forecast_dates[1:])

preds['Ticker'] = np.array(t).reshape(-1)
d = np.array(d).reshape(-1)
preds.index = d
preds

In [ ]:
# Graficación de una muestra de los datos predichos
trace1 = go.Scatter(
    x = df_c.index,
    y = df_c['AAPL'],
    mode = 'lines',
    name = 'TEST'
)
trace2 = go.Scatter(
    x = preds.index,
    y = preds[preds['Ticker']=='AAPL']['RF'],
    mode = 'lines',
    name = 'CNN'
)
layout = go.Layout(
    title = 'Stock',
    xaxis = {'title' : 'Date'},
    yaxis = {'title' : 'Values'}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()